In [ ]:
#!pip install requests
#!pip install dash

In [ ]:
#Załadowanie potrzebnych bibliotek
import requests
import pandas as pd
import io
import plotly.express as px
from dash import dcc, html, Dash, callback, Output, Input

#Pobranie danych OECD korzystając z API
url = 'https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_HW@DF_AVG_USL_WK_WKD,1.0/ROU+ZAF+BRA+BGR+HRV+CYP+MLT+AUS+AUT+BEL+CHL+COL+CRI+CZE+EST+FIN+FRA+DEU+GRC+HUN+IRL+ISR+ITA+KOR+LVA+LTU+LUX+MEX+NLD+NZL+POL+PRT+SVK+SVN+ESP+TUR+GBR+USA..._T._T...._T._T...?startPeriod=2011&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
whw = pd.read_csv(io.StringIO(response.text))

url = 'https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_EARNINGS@RMW/BRA+BGR+HRV+MLT+PER+ROU+RUS+SVN+SVK+ESP+TUR+GBR+USA+NLD+LVA+LTU+LUX+MEX+NZL+POL+PRT+DEU+GRC+HUN+IRL+ISR+JPN+KOR+AUS+BEL+CAN+CHL+COL+CRI+CZE+EST+FRA..USD_PPP.A...?startPeriod=2001&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
mw = pd.read_csv(io.StringIO(response.text))

url = 'https://sdmx.oecd.org/public/rest/data/OECD.ELS.SAE,DSD_TEMP@DF_TEMP_D,1.0/AUT+BEL+CAN+CHL+COL+CRI+CZE+EST+FIN+FRA+DNK+DEU+GRC+HUN+ISL+IRL+ITA+JPN+KOR+LVA+LTU+LUX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+BGR+HRV+CYP+MLT+ROU.EMP_TEMP+EMP_PERM.._T._T.ICSE93_1.A?startPeriod=2018&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
epj = pd.read_csv(io.StringIO(response.text))

#Przygotowanie danych do użycia w wykresach
whw = whw.rename(columns={'Reference area':'country'})
whw23 = whw.loc[whw['TIME_PERIOD'] == 2023]
whw = whw.sort_values(by='country')

epj = epj.rename(columns={'Reference area':'country', 'TIME_PERIOD':'time'})
epj = epj.sort_values(by=['country', 'time'], ascending=[True, False])
epj['OBS_VALUE'] = epj['OBS_VALUE']*1000

#Stworzenie wykresów
fig2 = px.choropleth(mw.sort_values(by='TIME_PERIOD'), locations='REF_AREA', color = 'OBS_VALUE', color_continuous_scale ='deep', hover_name='Reference area',
                    projection='natural earth', animation_frame='TIME_PERIOD', labels = {'OBS_VALUE': 'Value','REF_AREA': 'Reference area', 'TIME_PERIOD': 'Year'})
fig2.update_layout(margin=dict(l=0, r=0, b=0, t=0), width=1500, height=530)

fig1 = px.bar(round(whw23.sort_values(by=['OBS_VALUE', 'country']),0), x = 'country', y = 'OBS_VALUE', text_auto = True, 
              color = 'OBS_VALUE', color_continuous_scale ='deep', labels = {'OBS_VALUE': 'Hours', 'country': 'Country'})
fig1.update_layout(xaxis_rangeslider_visible=True)
fig1.update_layout(plot_bgcolor='#F9F9FD')
fig1.update_traces(textfont_size=9, textangle=0, textposition="outside", cliponaxis=False)
fig1.update_yaxes(range = [0, 50])
fig1.update_layout(height=540)


#Dashboard
app = Dash(__name__)
app.config.suppress_callback_exceptions=True

colors = {'background': '#97CCE0'}

app.layout = html.Div([
    html.Div([
      html.H1(children='LABOUR MARKET DASHBOARD', 
            style={'textAlign':'center', 'color': '#3F3A6D', 'font-family':"Perpetua", 'font-size': 64}),  
    ], style={'height': '55px'}
    ),
    html.H4(children='. ', style={'backgroundColor': colors['background'], 'color': colors['background']}),
    dcc.Tabs(id='tabs', value='tab1', children=[
        dcc.Tab(label='REAL MINIMUM WAGES AT CONSTANT PRICES', value='tab1'),
        dcc.Tab(label='AVERAGE USUAL WEEKLY HOURS WORKED ON THE MAIN JOB', value='tab2'),
        dcc.Tab(label='EMPLOYMENT BY PERMANENCY OF THE JOB', value='tab3')
    ], style={'font-family':"Perpetua", 'font-size': 19, "font-weight": "bold", 'color': '#38396B'}),
    html.Div(id='tabs-content'),
    html.H4(children='. ', style={'backgroundColor': colors['background'], 'color': colors['background']})
])

@callback(Output('tabs-content', 'children'),
         Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab1':
        return html.Div([
            html.H2(children='REAL MINIMUM WAGES AT CONSTANT PRICES', style={'textAlign':'center', 'font-family':"Perpetua", 'font-size': 36, 'color': '#3C6891'}) ,
            dcc.Graph(figure = fig2, id='map-content')
        ])
    elif tab == 'tab2':
        return html.Div([
            html.H2(children='AVERAGE USUAL WEEKLY HOURS WORKED ON THE MAIN JOB', style={'textAlign':'center', 'font-family':"Perpetua", 'font-size': 36,  'color': '#3C6891'}),
            html.Div([
                html.H3(children='Average usual weekly hours worked on the main job in 2023', 
                style={'width': '58%', 'display': 'inline-block', 'textAlign':'center', 'font-family':"Perpetua", 'font-size': 25}),
                html.H3(children="Average usual weekly hours worked on the main job in", 
                style={'width': '40%', 'display': 'inline-block', 'textAlign':'center', 'font-family':"Perpetua", 'font-size': 25})
            ]),
            html.Div([
                dcc.Graph(figure = fig1)
            ], style={'width': '58%', 'display': 'inline-block', 'padding': '0 20'}),
            html.Div([
                dcc.Dropdown(whw.country.unique(), 'Australia', 
                             style={'font-family':"Perpetua", 'font-size': 19, 'backgroundColor': '#F7FFEB'}, id='dropdown-selection'),
                dcc.Graph(id='graph-content')
            ], style={'width': '40%', 'display': 'inline-block', 'padding': '0 20'}) 
            ])
    elif tab == 'tab3':
        return html.Div([
            html.H2(children='EMPLOYMENT BY PERMANENCY OF THE JOB', style={'textAlign':'center', 'font-family':"Perpetua", 'font-size': 36,  'color': '#3C6891'}),
            html.Div([
                html.H3(children="Employment by permanency of the job in", 
                style={'textAlign':'center', 'font-family':"Perpetua", 'font-size': 25}),
                html.Div([
                    dcc.Dropdown(epj.country.unique(), 'Austria', 
                             style={'font-family':"Perpetua", 'font-size': 20, 'backgroundColor': '#F7FFEB'}, id='dropdown-selection2'),
                ], style={'width': '49%', 'display': 'inline-block'}),
                html.Div([
                dcc.Dropdown(epj.time.unique(), 2023, 
                             style={'font-family':"Perpetua", 'font-size': 18, 'backgroundColor': '#F7FFEB'}, id='dropdown-selection3')
                ], style={'width': '49%', 'display': 'inline-block', 'float': 'right'}),
                dcc.Graph(id='graph-content2')
            ])
        ])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)

def update_graph(value):
    dff = whw.loc[whw.country == value].sort_values(by='TIME_PERIOD')
    fig = px.line(dff, x= 'TIME_PERIOD', y = 'OBS_VALUE', 
           color_discrete_sequence=['#009C3D'], 
           labels = {'OBS_VALUE': 'Hours', 'TIME_PERIOD': 'Year'})
    fig.update_layout(plot_bgcolor='white')
    fig.update_yaxes(mirror=True, showline=True, linecolor='lightgrey', gridcolor='lightgrey')
    fig.update_xaxes(mirror=True, showline=True, linecolor='lightgrey', gridcolor='lightgrey')
    
    return fig

@callback(
    Output(component_id='graph-content2', component_property='figure'),
    Input(component_id='dropdown-selection2', component_property='value'),
    Input(component_id='dropdown-selection3', component_property='value')
)

def update_graph(value2,value3):
    dff = epj.loc[(epj.country == value2) & (epj.time == value3)].sort_values(by=['country', 'time'])
    fig = px.pie(dff, values = 'OBS_VALUE', names = 'Measure', hole=.5, color_discrete_sequence=['#54A7AD', '#ECF7C3'],
                labels = {'OBS_VALUE': 'Persons'})
    fig.update_traces(textinfo='value')
    fig.update_layout(margin=dict(t=40, b=10, l=10, r=40),  
                      legend = dict(font = dict(family = "Perpetua", size = 25, color = "black"), 
                      yanchor="top", y=0.70, xanchor="right", x=0.92))
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode='external')